The retrievement of articles metadata (this example is for articles in the machine learning category) seems to be limited to 50000 results.

To download in bulk we have to use https://arxiv.org/help/oa/index

In [1]:
import requests
import json
import atoma
url = 'http://export.arxiv.org/api/query?search_query=cat:cs.LG&start=10&max_results=10'
response = requests.get(url)
feed = atoma.parse_atom_bytes(response.content)
print(feed)

AtomFeed(title=AtomTextConstruct(text_type=<AtomTextType.html: 'html'>, lang=None, value='ArXiv Query: search_query=cat:cs.LG&id_list=&start=10&max_results=10'), id_='http://arxiv.org/api/1K0+JJNvmhpwFqUDN6Sxt+rR4xI', updated=datetime.datetime(2020, 10, 10, 0, 0, tzinfo=tzoffset(None, -14400)), authors=[], contributors=[], links=[AtomLink(href='http://arxiv.org/api/query?search_query%3Dcat%3Acs.LG%26id_list%3D%26start%3D10%26max_results%3D10', rel='self', type_='application/atom+xml', hreflang=None, title=None, length=None)], categories=[], generator=None, subtitle=None, rights=None, icon=None, logo=None, entries=[AtomEntry(title=AtomTextConstruct(text_type=<AtomTextType.text: 'text'>, lang=None, value='Evaluation of the Performance of the Markov Blanket Bayesian Classifier\n  Algorithm'), id_='http://arxiv.org/abs/cs/0211003v1', updated=datetime.datetime(2002, 11, 1, 18, 9, 56, tzinfo=tzutc()), authors=[AtomPerson(name='Michael G. Madden', uri=None, email=None)], contributors=[], link

In [2]:
print(len(feed.entries))

10


In [3]:
print(feed.entries[0].title.value)

Evaluation of the Performance of the Markov Blanket Bayesian Classifier
  Algorithm


In [4]:
print(feed.entries[0].authors)

[AtomPerson(name='Michael G. Madden', uri=None, email=None)]


In [5]:
print(feed.entries[0].published)

2002-11-01 18:09:56+00:00


In [6]:
print(feed.entries[0].summary.value)

The Markov Blanket Bayesian Classifier is a recently-proposed algorithm for
construction of probabilistic classifiers. This paper presents an empirical
comparison of the MBBC algorithm with three other Bayesian classifiers: Naive
Bayes, Tree-Augmented Naive Bayes and a general Bayesian network. All of these
are implemented using the K2 framework of Cooper and Herskovits. The
classifiers are compared in terms of their performance (using simple accuracy
measures and ROC curves) and speed, on a range of standard benchmark data sets.
It is concluded that MBBC is competitive in terms of speed and accuracy with
the other algorithms considered.


In [7]:
print(feed.entries[0])

AtomEntry(title=AtomTextConstruct(text_type=<AtomTextType.text: 'text'>, lang=None, value='Evaluation of the Performance of the Markov Blanket Bayesian Classifier\n  Algorithm'), id_='http://arxiv.org/abs/cs/0211003v1', updated=datetime.datetime(2002, 11, 1, 18, 9, 56, tzinfo=tzutc()), authors=[AtomPerson(name='Michael G. Madden', uri=None, email=None)], contributors=[], links=[AtomLink(href='http://arxiv.org/abs/cs/0211003v1', rel='alternate', type_='text/html', hreflang=None, title=None, length=None), AtomLink(href='http://arxiv.org/pdf/cs/0211003v1', rel='related', type_='application/pdf', hreflang=None, title='pdf', length=None)], categories=[AtomCategory(term='cs.LG', scheme='http://arxiv.org/schemas/atom', label=None), AtomCategory(term='I.2.6', scheme='http://arxiv.org/schemas/atom', label=None)], published=datetime.datetime(2002, 11, 1, 18, 9, 56, tzinfo=tzutc()), rights=None, summary=AtomTextConstruct(text_type=<AtomTextType.text: 'text'>, lang=None, value='The Markov Blanket 

Using the sickle module to obtain arxiv's metadata through the OAI_PMH protocol

In [29]:
from sickle import Sickle
from requests.exceptions import HTTPError
import re
import time

How many machine learning articles in 2020 so far?

In [36]:
url = 'http://export.arxiv.org/oai2'
count = 0
token = None
while 1:
    print('TOKEN: ' + str(token))
    sickle = Sickle(url)
    if not token:
        records = sickle.ListRecords(**{'metadataPrefix': 'arXiv',
                                        'set': 'cs',
                                        'ignore_deleted': True,
                                        'from': '2020-01-01'})
    else:
        records = sickle.ListRecords(**{'resumptionToken': token})   
    try:
        for r in records:
            if 'cs.LG' in r.metadata['categories'][0] or 'stat.ML' in r.metadata['categories'][0]:
                #print(r.metadata)
                count = count + 1
    except HTTPError as e:
        if e.response.status_code == 503:
            m = re.search('resumptionToken=.+&', str(e).split(' ')[-1])
            token = m.group(0)[:-1].replace('resumptionToken=', '').replace('%7C', '|')
            time.sleep(5)
        else:
            raise e
    
    print(count)

TOKEN: None
503 Server Error: Retry after specified interval for url: http://export.arxiv.org/oai2?resumptionToken=4951178%7C1001&verb=ListRecords
128
TOKEN: 4951178|1001
503 Server Error: Retry after specified interval for url: http://export.arxiv.org/oai2?resumptionToken=4951178%7C2001&verb=ListRecords
324
TOKEN: 4951178|2001
503 Server Error: Retry after specified interval for url: http://export.arxiv.org/oai2?resumptionToken=4951178%7C4001&verb=ListRecords
912
TOKEN: 4951178|4001
503 Server Error: Retry after specified interval for url: http://export.arxiv.org/oai2?resumptionToken=4951178%7C6001&verb=ListRecords
1601
TOKEN: 4951178|6001
503 Server Error: Retry after specified interval for url: http://export.arxiv.org/oai2?resumptionToken=4951178%7C8001&verb=ListRecords
2450
TOKEN: 4951178|8001
503 Server Error: Retry after specified interval for url: http://export.arxiv.org/oai2?resumptionToken=4951178%7C9001&verb=ListRecords
2802
TOKEN: 4951178|9001
503 Server Error: Retry after s

HTTPError: 503 Server Error: Retry after specified interval for url: http://export.arxiv.org/oai2?resumptionToken=4951178%7C67001&verb=ListRecords